## Running LOCAL spark to read and write to HDFS

Local spark runs the drivers and executors in the same JVM. As such it has access to the Kerberos ticket in the cache. 

Hence we can skip the task of producing a HDFS Delegation token which is produced internally. Local Spark can read write to HDFS without any extra development similar to how we can do the same with EMR Spark (with driver running in client mode) as long as the driver runs inside a workspace with a valid Kerberos ticket (you can examine it by running `kilist` in the terminal)

In [1]:
!klist

Ticket cache: FILE:/tmp/krb5cc_12574
Default principal: dominospark@KDCDOMINO.COM

Valid starting       Expires              Service principal
08/16/2021 12:12:32  08/16/2021 22:12:32  krbtgt/KDCDOMINO.COM@KDCDOMINO.COM
	renew until 08/17/2021 12:12:32


In [1]:
import os

%env HDFS_PATH /user/dominospark/small_data
%env HADOOP_HOME /usr/lib/hadoop
%env HADOOP_CONF_DIR /etc/hadoop/conf
%env HADOOP_YARN_HOME /usr/lib/hadoop
%env HADOOP_MAPRED_HOME /usr/lib/hadoop
%env HADOOP_HDFS_HOME /usr/lib/hadoop-hdfs

%env SPARK_HOME /usr/lib/spark
%env SPARK_CONF_DIR /etc/spark/conf
#%env PYTHONPATH /opt/spark/python/lib/py4j-0.10.7-src.zip
%env PYSPARK_PYTHON /opt/conda/bin/python
%env PYSPARK_DRIVER_PYTHON /opt/conda/bin/python
hdfs_endpoint=os.environ['HDFS_ENDPOINT']

env: HDFS_PATH=/user/dominospark/small_data
env: HADOOP_HOME=/usr/lib/hadoop
env: HADOOP_CONF_DIR=/etc/hadoop/conf
env: HADOOP_YARN_HOME=/usr/lib/hadoop
env: HADOOP_MAPRED_HOME=/usr/lib/hadoop
env: HADOOP_HDFS_HOME=/usr/lib/hadoop-hdfs
env: SPARK_HOME=/usr/lib/spark
env: SPARK_CONF_DIR=/etc/spark/conf
env: PYSPARK_PYTHON=/opt/conda/bin/python
env: PYSPARK_DRIVER_PYTHON=/opt/conda/bin/python


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, DoubleType, IntegerType
import random

In [3]:
sparkSession = SparkSession.builder.appName("example-pyspark-read-and-write") \
.master('local[*]') \
.config("fs.default.name", hdfs_endpoint) \
.getOrCreate()

In [7]:
sc=sparkSession.sparkContext

In [8]:
def inside(p):
    x, y = random.random(), random.random()
    return x*x + y*y < 1


In [9]:


columns = StructType([ StructField("name", StringType(), True),
                      StructField("value", DoubleType(), True)
                    ])

count = sc.parallelize(range(0, 1000),1) \
             .filter(inside).count()
data = [("Pi",4.0 * count/1000)]

df = sparkSession.createDataFrame(data=data, schema=columns)

df.show()

+----+-----+
|name|value|
+----+-----+
|  Pi|3.104|
+----+-----+



In [10]:
#Let us write to a dataset
ds_path = '/user/dominospark/my-pi'
!/mnt/code/scripts/my_hdfs.sh dfs -rmr '/user/dominospark/my-pi*'
df.write.csv(ds_path)
#Read it back
sparkSession.read.csv(ds_path).show()


rmr: DEPRECATED: Please use '-rm -r' instead.
Deleted /user/dominospark/my-pi
+---+-----+
|_c0|  _c1|
+---+-----+
| Pi|3.104|
+---+-----+



In [14]:
!/mnt/code/scripts/my_hdfs.sh dfs -rmr '/user/dominospark/small-data-100/'
hdfs_src_path = '/user/dominospark/largedata/'
hdfs_dest_path =  '/user/dominospark/small-data-100/'
local_dest_path = 'file:///mnt/data/ON-DEMAND-SPARK/small-data-100'
!rm -rf /mnt/data/ON-DEMAND-SPARK/small-data-100
filter_criteria = 100
sparkSession = SparkSession.builder.appName("Generate Data") \
    .config("spark.dynamicAllocation.enabled", "false") \
    .config("fs.default.name", hdfs_endpoint) \
    .getOrCreate()

    
columns = StructType([ StructField("id", IntegerType(), True), \
                       StructField("v1", IntegerType(), True),\
                       StructField("v2", IntegerType(), True),\
                       StructField("v3", IntegerType(), True) ])

df_load = sparkSession.read.csv(hdfs_src_path,columns)
df_load_filtered = df_load.where(df_load.id < filter_criteria)
df_load_filtered.write.csv(hdfs_dest_path)
df_load_filtered.write.csv(local_dest_path)


rmr: DEPRECATED: Please use '-rm -r' instead.
Deleted /user/dominospark/small-data-100


In [8]:
df_load_read = sparkSession.read.csv(hdfs_dest_path,columns)
df_load_read.show()

+---+---+---+---+
| id| v1| v2| v3|
+---+---+---+---+
|  0| 63| 94| 50|
|  1| 25| 26| 73|
|  2| 84| 84| 84|
|  3| 47| 19|  8|
|  4| 24| 31|  6|
|  5| 75| 17| 11|
|  6| 49| 38| 57|
|  7| 56| 31| 90|
|  8|100|  3|  3|
|  9| 43| 72| 34|
| 10| 18| 64| 57|
| 11| 63| 75| 80|
| 12| 82| 85| 28|
| 13| 31|  8| 42|
| 14| 20| 80|  3|
| 15| 27| 91| 86|
| 16| 55| 70| 42|
| 17| 69|  3|  5|
| 18| 65| 28| 28|
| 19| 57|  8| 69|
+---+---+---+---+
only showing top 20 rows



In [10]:
df_load_read = sparkSession.read.csv(local_dest_path,columns)
df_load_read.show()

+---+---+---+---+
| id| v1| v2| v3|
+---+---+---+---+
|  0| 63| 94| 50|
|  1| 25| 26| 73|
|  2| 84| 84| 84|
|  3| 47| 19|  8|
|  4| 24| 31|  6|
|  5| 75| 17| 11|
|  6| 49| 38| 57|
|  7| 56| 31| 90|
|  8|100|  3|  3|
|  9| 43| 72| 34|
| 10| 18| 64| 57|
| 11| 63| 75| 80|
| 12| 82| 85| 28|
| 13| 31|  8| 42|
| 14| 20| 80|  3|
| 15| 27| 91| 86|
| 16| 55| 70| 42|
| 17| 69|  3|  5|
| 18| 65| 28| 28|
| 19| 57|  8| 69|
+---+---+---+---+
only showing top 20 rows



In [11]:
sparkSession.stop()

In [12]:
!/mnt/code/scripts/my_hdfs.sh dfs -ls

Found 11 items
drwxr-xr-x   - dominospark dominospark          0 2021-08-14 18:53 .sparkStaging
drwxr-xr-x   - dominospark dominospark          0 2021-08-13 20:45 example
drwxr-xr-x   - dominospark dominospark          0 2021-08-13 21:45 large-data
drwxr-xr-x   - dominospark dominospark          0 2021-08-14 18:55 largedata
drwxr-xr-x   - dominospark dominospark          0 2021-08-13 21:57 ld-10
drwxr-xr-x   - dominospark dominospark          0 2021-08-14 19:27 my-pi
drwxr-xr-x   - dominospark dominospark          0 2021-08-13 21:07 mypi
drwxr-xr-x   - dominospark dominospark          0 2021-08-13 21:58 sd-5
drwxr-xr-x   - dominospark dominospark          0 2021-08-14 19:27 small-data-100
drwxr-xr-x   - dominospark dominospark          0 2021-08-14 19:22 smalldata-10
drwxr-xr-x   - dominospark dominospark          0 2021-08-14 18:53 smalldata-1000


'/opt/spark'